# Learning 1d Pure Advection in Conservative Form

To verify that our PINN implementation is working well, we test the PINN on the following problem:
$$
    \begin{cases}
        u_t + a(t)u_x = 0\\
        \lim_{|x|\rightarrow\infty}u(t, x) = 0\\
        u(0, x) = u_0(x)
    \end{cases}
$$ where $u_0 = \mathcal{N}(x|0, 1)$. 

In this case, the NN solution is assumed to have the following model (conservative advection):
$$
    \frac{\partial u_{\theta_1}}{\partial t} + \frac{\partial}{\partial x}\bigg(\mathcal{V}_{\theta_2}(t)u_{\theta_1}\bigg) = 0
$$

In this test case, the true advection velocity is a time-varying function, no spatial dependence. Furthermore, since the coefficient neural network is a time-dependent function, the conservative and non-conservative form of the PDE coincide. In this notebook, we generate data from such a PDE and test the PINN learning in both conservative mode and non-conservative mode.

In this note, we use the Fourier embedded deep neural nets to overcome the high frequency in time domain.

In [ ]:
from PINN.AdvectionNet import *
# Testing
import matplotlib.pyplot as plt
import torch
import numpy as np
import scipy

# set random seeds
np.random.seed(10)
torch.manual_seed(10);

import numpy as np    
import matplotlib.pyplot as plt

import time
import pylab as pl
from IPython import display
from IPython.display import clear_output

# Experiment 1: High Frequency with Vanilla NN

Consider the following PDE, taken from: https://scullen.com.au/DSc/Publications/scullen_92.pdf (Page 44)

$$
    \frac{\partial u}{\partial t} + v_0\sin(\omega t)\frac{\partial u}{\partial x} = 0
$$ which admits an analytical solution:
$$
    u(t,x) = \exp\bigg(
        -k[x - x_0 - 2v_0\omega^{-1}\sin^2(\frac{1}{2}\omega t)]^2
    \bigg)
$$ with parameters $k, x_0$ from the initial condition.

In [ ]:
# parameters
x0 = 2.0
v0 = 2.0
k = 5.0
omega = 2.0*np.pi
# time grid
t_start = 0.0
t_end = 2*np.pi
dt = 0.001
tgrid = np.arange(t_start, t_end, dt)
nt = len(tgrid)
# spatial grid
x_left, x_right = 0.0, 5.0
dx = 0.005
xgrid = np.arange(x_left, x_right, dx)
nx = len(xgrid)

# solution
u_sol = np.zeros([nt, nx])
for i in range(nt):
    t = tgrid[i]
    u_sol[i, :] = np.exp(-k * (( xgrid - x0 ) - (2*v0/omega) * (np.sin(0.5*omega*t) ** 2)) ** 2 )

In [ ]:
# plot velocity as a function of time
plt.figure(2);
plt.plot(tgrid, v0*np.sin(omega*tgrid), "--", color="blue", alpha=0.6, lw=4.0);
plt.xlabel(r"$t$"); plt.ylabel(r"Velocity (High Frequency)");

In [ ]:
# save data for PINN training
scipy.io.savemat("../data/LinearOscillator/Time_Variable_Advection.mat", {
    "xgrid": xgrid.reshape(1, -1),
    "tgrid": tgrid.reshape(1, -1),
    "pmc": u_sol
})

# save a smaller dataset
subsample_t = 6
subsample_x = 2
xgrid_small = xgrid.reshape(1, -1)[:, 0:-1:subsample_x].squeeze(),
tgrid_small = tgrid.reshape(1, -1)[:, 0:-1:subsample_t].squeeze()
u_sol_small = u_sol[0:-1:subsample_t, 0:-1:subsample_x]
scipy.io.savemat("../data/LinearOscillator/Time_Variable_Advection_small.mat", {
    "xgrid": xgrid_small,
    "tgrid": tgrid_small,
    "pmc": u_sol_small
})
print("Training data size = {}\n".format(u_sol_small.shape))

In [ ]:
# PINN experiments
data_path = "../data/LinearOscillator/Time_Variable_Advection_small.mat"
# create PINN
pinn = AdvectionNet(indim=2, outdim=1, data_path=data_path, scheduler="ExponentialLR", coef_mode="time")
# testing training
info = train(
    pinn, pinn.optimizer, pinn.scheduler, batch_size=2**10,
    epochs=200, batch_print=200, mode="all", conservative_pde=True
)

In [ ]:
inputs = cartesian_data(pinn.tgrid, pinn.xgrid)
G_pred = pinn.G_nn(inputs[:, 0][:, None]).reshape([pinn.nx, pinn.nt]).detach().numpy().T
# spatial and temporal grid
tgrid = pinn.tgrid.detach().numpy()
xgrid = pinn.xgrid.detach().numpy()

In [ ]:
plt.plot(tgrid, G_pred[:, 0], color="red", lw=2.0, label="predicted");
plt.plot(tgrid, v0*np.sin(omega*tgrid), "--", color="blue", alpha=0.6, lw=4.0);
plt.title("Inferred advection coefficient as a function of time (Fourier Net)");
plt.legend();